All Pretrained models needed for this project are already  via pip install, no need for any external connections

In [ ]:
!pip install sentence_transformers
!pip install nltk
!pip install spacy
!pip install flair
!pip install PyDictionary
!pip install torch

In [ ]:
!pip install language-tool-python
!pip install wordfreq
!python -m spacy download en_core_web_sm
!python -m spacy download en_core_web_md
# !python -m spacy download en_core_web_lg
!pip install transformers sentencepiece sacremoses

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from transformers import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import namedtuple
from nltk import word_tokenize
import nltk
from nltk.corpus import brown
from nltk import word_tokenize
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('omw-1.4')
import spacy
import flair
from wordfreq import zipf_frequency
from nltk.corpus import stopwords

nlp_small = spacy.load("en_core_web_sm")
nlp_medium = spacy.load("en_core_web_md")
# nlp_large = spacy.load("en_core_web_lg")
from spacy.lang.en.stop_words import STOP_WORDS

import string
Dataset = namedtuple('Dataset', 'name, train, test')
Model = namedtuple('Model', 'type, name, dimension, corpus, model')


In [ ]:
# !wget https://www.inf.uni-hamburg.de/en/inst/ab/lt/resources/data/complex-word-identification-dataset/cwishareddataset.zip
# !unzip cwishareddataset.zip

In [ ]:
# pd.set_option('display.max_columns', 500)
# pd.set_option('display.max_colwidth', 200)
# MAIN_PATH_DATASET = "traindevset/english/"
# genres = ['Wikipedia', 'WikiNews', 'News']
# datasets = ['Train', 'Dev']
# columns = ['id', 'sentence', "start", "end", "target", 
#            "nat", "non_nat", "nat_marked", "non_nat_marked", "binary", "prob"]


# datasets = [Dataset('Wikipedia', 'Train', 'Dev'),
#             Dataset('WikiNews', 'Train', 'Dev'),
#             Dataset('News', 'Train', 'Dev')]

# feature_categories = []

# def load_df(path):
#     df = pd.read_csv(path, header=None, sep = "\t")
#     df.columns = columns
#     return df

# datasets = [Dataset(d.name, load_df(MAIN_PATH_DATASET + d.name + '_' + d.train + '.tsv'),
#                             load_df(MAIN_PATH_DATASET + d.name + '_' + d.test + '.tsv'))
#                             for d in datasets]

In [ ]:
import copy

In [ ]:
model_ST_bert = SentenceTransformer('bert-base-nli-mean-tokens')

In [ ]:
from transformers import BartForConditionalGeneration
# models we gonna use for this project

model_ctrl44 = BartForConditionalGeneration.from_pretrained("liamcripwell/ctrl44-simp")
tokenizer_ctrl44 = AutoTokenizer.from_pretrained("liamcripwell/ctrl44-simp")

model_pega = PegasusForConditionalGeneration.from_pretrained("tuner007/pegasus_paraphrase")
tokenizer_pega = PegasusTokenizerFast.from_pretrained("tuner007/pegasus_paraphrase")

tokenizer_paws = AutoTokenizer.from_pretrained("Vamsi/T5_Paraphrase_Paws")
model_paws = AutoModelForSeq2SeqLM.from_pretrained("Vamsi/T5_Paraphrase_Paws")

tokenizer_parr = AutoTokenizer.from_pretrained("prithivida/parrot_paraphraser_on_T5")
model_parr = AutoModelForSeq2SeqLM.from_pretrained("prithivida/parrot_paraphraser_on_T5")

In [ ]:
def get_paraphrased_sentences(model, tokenizer, sentence, num_return_sequences=5, num_beams=5, max_length=128):
  # tokenize the text to be form of a list of token IDs
  inputs = tokenizer([sentence], truncation=True, padding="longest", return_tensors="pt")
  # generate the paraphrased sentences
  outputs = model.generate(
    **inputs,
    num_beams=num_beams,
    num_return_sequences=num_return_sequences,
    max_length=max_length
  )
  # decode the generated sentences using the tokenizer to get them back to text
  return tokenizer.batch_decode(outputs, skip_special_tokens=True)

In [ ]:
def get_similar_words(word_used, tag):
  zip_scores_dict = {}
  sim_scores_dict = {}
  syn_list = extract_synonyms(word_used, tag)  # SynSets nltk
  # hyper_list = extract_hypernyms(word_used, tag) 
  for syn in syn_list:
    print(syn)
    tokens = nlp_medium(word_used + " " + syn)
    token1 = tokens[0]
    token2 = tokens[1]
    sim_score = token1.similarity(token2)
    if syn != word_used: 
      zip_scores_dict[syn] = zipf_frequency(syn, 'en')
      sim_scores_dict[syn] = sim_score
      print(syn, ": zipf_score= ", zipf_frequency(syn, 'en'), ", sim_score= ", sim_score)
  
  sim_scores_dict = sort_dict(sim_scores_dict)
  zip_scores_dict = sort_dict(zip_scores_dict)

  highest_group_words = []
  ind = len(sim_scores_dict) - 1
  print(ind)
  best_sim_score = sim_scores_dict[list(sim_scores_dict.keys())[ind]]
  highest_group_words.append(list(sim_scores_dict.keys())[ind])
  flag = 1
  while flag == 1 and ind >= 0:
    ind = ind - 1
    if sim_scores_dict[list(sim_scores_dict.keys())[ind]] == best_sim_score:
      highest_group_words.append(list(sim_scores_dict.keys())[ind])
    else:
      flag = 0

  if len(highest_group_words) == 1:
    return highest_group_words[0]
  else:
    flag = 1
    ind = len(zip_scores_dict)
    while flag == 1 and ind >= 0:
      ind = ind - 1
      if list(sim_scores_dict.keys())[ind] in highest_group_words:
        return list(sim_scores_dict.keys())[ind]
    return list(zip_scores_dict.keys())[-1]

  #cos_sims_bert = get_cos_similarities(syn_list, word_used)



In [ ]:
## Using NLTK SynSets

def extract_synonyms(word, p):
  synonyms = []

  # part of speech
  if (p == "verb"):
    syns = wordnet.synsets(word, pos = wordnet.VERB)
  if (p == "noun"):
    syns = wordnet.synsets(word, pos = wordnet.NOUN)
  if (p == "adj"):
    syns = wordnet.synsets(word, pos = wordnet.ADJ)
    
  for syn in syns:
    for lem in syn.lemmas():
      synonyms.append(lem.name())
  
  return(list(set(synonyms)))

def extract_hypernyms(word, p):
	hypernyms = []

	# part of speech
	if (p == "verb"):
		syns2 = wordnet.synsets(word, pos = wordnet.VERB)
	if (p == "noun"):
		syns2 = wordnet.synsets(word, pos = wordnet.NOUN)
	if (p == "adj"):
		syns2 = wordnet.synsets(word, pos = wordnet.ADJ)
	

	for syn in syns2:
		for hyper in syn.hypernyms():
			hypernyms.append(hyper)
	
	return(list(set(hypernyms)))

def extract_hyponyms(word, p):
  hyponyms = []

  # part of speech
  if (p == "verb"):
    syns2 = wordnet.synsets(word, pos = wordnet.VERB)
  if (p == "noun"):
    syns2 = wordnet.synsets(word, pos = wordnet.NOUN)
  if (p == "adj"):
    syns2 = wordnet.synsets(word, pos = wordnet.ADJ)
    
  for syn in syns2:
    for hypo in syn.hyponyms():
      hyponyms.append(hypo)
  
  return(list(set(hyponyms)))

def shortest_hypernym_distances(word, p):
  distances = []
  shortest = 99999999
  result = []
  
  # part of speech
  if (p == "verb"):
    syns3 = wordnet.synsets(word, pos = wordnet.VERB)
  if (p == "noun"):
    syns3 = wordnet.synsets(word, pos = wordnet.NOUN)
  if (p == "adj"):
    syns3 = wordnet.synsets(word, pos = wordnet.ADJ)

  for syn in syns3:
    for path in syn.hypernym_paths():
      if len(path) < shortest:
        shortest = min([len(path) for path in syn.hypernym_paths()])
        result = [path]
    distances.append(min([len(path) for path in syn.hypernym_paths()]))
  
  if(result):
    return result;
  print("ERROR - no shortest hypernym distances.");

#display(shortest_hypernym_distances("love", "noun"))

In [ ]:
main_POS_groups = ["noun", "verb", "adj"]
SUBJECT_PRONOUNS = ["I", "you", "he", "she", "we", "they"]
OBJECT_PRONOUNS = ["me", "you", "him", "her", "us", "them", "whom"]
POSS_PRONOUNS = ["mine", "ours", "yours", "his", "hers", "its", "theirs"]
POSS_DETS = ["my", "our", "your", "his", "her", "its", "their"]
# special rule to keep determiners for nouns
MAIN_DETS = ["a", "an", "the", "some"]
MAIN_POS_TAGS = {"JJ":	"adj", "JJR": "adj", "JJS": "adj", "NNP":	"propernoun", 
            "NNPS": "propernoun", "RB": "adverb", "RBR": "adverb", "RBS": "adverb",
            "VB": "verb", "VBD": "verb", "VBG": "verb", "VBN": "verb", "VBP": "verb", "VBZ": "verb",
            "WRB": "adverb", "NN": 'noun', 'NNS': 'noun'
# 33.	WDT	Wh-determiner
# 34.	WP	Wh-pronoun
# 35.	WP$	Possessive wh-pronoun
# 36.	WRB	Wh-adverb
}

In [ ]:
sentences = [
    "Three years later, the coffin was still full of Jello.",
    "The fish contemplated escaping the fishbowl and into the toilet where he saw his friend go.",
    "The person box was packed with jelly many dozens of months later.",
    "The way the building was designed seemed counterintuitive to most onlookers.",
    "They found many egregious mistakes in his calculations.",
    "He found a gargantuan walnut in his flask.",
    "There aren't any more bananas",
    "David and his prolixity"
]

def get_pos_tags_list(s):
  pos_sentences = []
  text = word_tokenize(s)
  pos_tags = nltk.pos_tag(text)
  pos_sentences.append(pos_tags)
  return pos_sentences

for s in sentences:
  print(get_pos_tags_list(s))
  


In [ ]:
import language_tool_python
pylangTool = language_tool_python.LanguageTool('en-US')
from PyDictionary import PyDictionary
dictionary=PyDictionary("hotel","ambush","nonchalant","perceptive")
print(dictionary.printMeanings()) 
print(dictionary.getMeanings()) 

words = set(nlp_small.vocab.strings)
word = 'would'
print(f"Is '{word}' an English word: {word in words}") 

In [ ]:
dictionary=PyDictionary("hotel","ambush","nonchalant","perceptive")
print(dictionary.printMeanings()) 
print(dictionary.getMeanings()) 

dictionary=PyDictionary("blithe")
print(dictionary.printMeanings()) 
print(dictionary.getMeanings()) 

In [ ]:
text = "Your the best but their are allso  good !"
matches = pylangTool.check(text)
len(matches)

print(matches[0])


text = "appeared disappear"
matches = pylangTool.check(text)
len(matches)
print(matches)



In [ ]:
text = "His natural reticence seemed under the influence of alcohol to disappear"
matches = pylangTool.check(text)
print("num of errors: ", len(matches))

dd = pylangTool.correct("His natural reticence seemed under the influence of alcohol to disappear")
print(dd)

In [ ]:
tokens = nlp_medium("huge gigantic enormous big")
  
for token in tokens:
    # Printing the following attributes of each token.
    # text: the word string, has_vector: if it contains
    # a vector representation in the model, 
    # vector_norm: the algebraic norm of the vector,
    # is_oov: if the word is out of vocabulary.
    print(token.text, token.has_vector, token.vector_norm, token.is_oov)
  

for t1 in range(0, len(tokens)):
  token1 = tokens[t1]
  for t2 in range(t1, len(tokens)):
    token2 = tokens[t2]
    print("Similarity: {}, {}".format(token1, token2), token1.similarity(token2))


In [ ]:
stop_words_nltk = set(stopwords.words('english'))

def get_important_word_stats(sent, z_freqs_Test):
  arr = list(z_freqs_Test.values())
  print('median=', np.median(arr))
  npquart1 = np.quantile(arr, 0.25)
  print('1st quartile=', npquart1)

  z_freqs_Test = sort_dict(z_freqs_Test)
  zkeys = []
  zvalues = []
  for k,v in z_freqs_Test.items():
    zkeys.append(k)
    zvalues.append(v)

  ### hard_constraint = requires sentence to retain more context words
  hard_constraint_set = zkeys[0: len(zkeys)//2]
  soft_cut = -1
  for vv in zvalues:
    if vv <= npquart1:  ## below the 1st quartile
      soft_cut = soft_cut + 1

  soft_constraint_set = zkeys[0: soft_cut]
  return hard_constraint_set, soft_constraint_set

In [ ]:
test_sentence = "David likes eating peanut butter with a spoon rather than a fork"

print(" ")

set_1 = set(get_paraphrased_sentences(model_pega, tokenizer_pega, test_sentence, num_beams=20, num_return_sequences=20))
set_2 = set(get_paraphrased_sentences(model_paws, tokenizer_paws, test_sentence, num_beams=20, num_return_sequences=20))
set_3 = set(get_paraphrased_sentences(model_parr, tokenizer_parr, test_sentence, num_beams=20, num_return_sequences=20))

combined_set = set_1.union(set_2).union(set_3) 
for x in combined_set:
  print(x)

In [ ]:
def easy_simplify(test_sentence):
  set_1 = set(get_paraphrased_sentences(model_pega, tokenizer_pega, test_sentence, num_beams=20, num_return_sequences=20))
  set_2 = set(get_paraphrased_sentences(model_paws, tokenizer_paws, test_sentence, num_beams=20, num_return_sequences=20))
  set_3 = set(get_paraphrased_sentences(model_parr, tokenizer_parr, test_sentence, num_beams=20, num_return_sequences=20))
  combined_set = set_1.union(set_2).union(set_3) 
  return combined_set

# combined_set_ex = easy_simplify("Dave's long-windedness was slowing down the meeting")
# for x in combined_set_ex:
#   print(x)

In [ ]:
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import PorterStemmer
snow_stemmer = SnowballStemmer("english")
from PyDictionary import PyDictionary
dictionary=PyDictionary("prolixity")
print(dictionary.printMeanings()) 
print(dictionary.getMeanings()) 

nlp_medium_words = set(nlp_medium.vocab.strings)
word_used = 'prolixity'
print(f"Is '{word_used}' an English word: {word_used in nlp_medium_words}") 

print(snow_stemmer.stem("generalized"))
print(snow_stemmer.stem("generalization"))
print(snow_stemmer.stem("hating"))

port_stemmer = PorterStemmer()
  
# choose some words to be stemmed
words = ["program", "programs", "programmer", "programming", "programmers", "hating"]
  
for w in words:
    print(w, " : ", port_stemmer.stem(w))



The real project to be used start here:

In [ ]:
def remove_punct(sent):
  test_str = sent.translate(str.maketrans('', '', string.punctuation))
  st_arr = test_str.split()
  curr_sent_no_punct = st_arr[0]
  for i in range(1,len(st_arr)):
    curr_sent_no_punct = curr_sent_no_punct + " " + st_arr[i]
  return curr_sent_no_punct

In [ ]:
import collections
def get_word_counts(sent):
  words = remove_punct(sent).lower().split()
  word_counts = collections.Counter(words)
  return word_counts

In [ ]:
nltk.download('cmudict')
from nltk.corpus import cmudict
cmudic = cmudict.dict()

def nsyllables(word):
    try:
        h = [len(list(y for y in x if y[-1].isdigit())) for x in cmudic[word.lower()]]
        return h[0]
    except KeyError:
        #if word not found in cmudict
        return 

def syllables(word):
    count = 0
    vowels = 'aeiouy'
    word = word.lower()
    if word[0] in vowels:
        count +=1
    for index in range(1,len(word)):
        if word[index] in vowels and word[index-1] not in vowels:
            count +=1
    if word.endswith('e'):
        count -= 1
    if word.endswith('le'):
        count += 1
    if count == 0:
        count += 1
    return count

In [ ]:
def break_sentences(text):
    doc = nlp_small(text)
    return list(doc.sents)
 
def word_count(text):
    sentences = break_sentences(text)
    words = 0
    for sentence in sentences:
        words += len([token for token in sentence])
    return words
 
def sentence_count(text):
    sentences = break_sentences(text)
    return len(sentences)
 
def avg_sentence_length(text):
    words = word_count(text)
    sentences = sentence_count(text)
    average_sentence_length = float(words / sentences)
    return average_sentence_length
 
def syllables_count(word):
    return nsyllables(word)
 
# Return total Difficult Words in a text
def difficult_words(text):
     
    nlp = spacy.load('en_core_web_sm')
    doc = nlp(text)
    # Find all words in the text
    words = []
    sentences = break_sentences(text)
    for sentence in sentences:
        words += [str(token) for token in sentence]
 
    # difficult words are those with syllables >= 2
    # easy_word_set is provide by Textstat as
    # a list of common words
    diff_words_set = set()
     
    for word in words:
        syllable_count = syllables_count(word)
        if word not in stop_words_nltk and syllable_count >= 2:
            diff_words_set.add(word)
 
    return len(diff_words_set)
 
# A word is polysyllablic if it has more than 3 syllables
# this functions returns the number of all such words
# present in the text
def poly_syllable_count(text):
    count = 0
    words = []
    sentences = break_sentences(text)
    for sentence in sentences:
        words += [token for token in sentence]
    for word in words:
        syllable_count = syllables_count(word)
        if syllable_count >= 3:
            count += 1
    return count

def gunning_fog(text):
    per_diff_words = (difficult_words(text) / word_count(text) * 100) + 5
    grade = 0.4 * (avg_sentence_length(text) + per_diff_words)
    return grade
 
 

In [ ]:
def concat_all_words(arr):
  ans = ""
  for i in arr:
    # concatenating the strings
    # using + operator
    ans = ans + " " + i
  return ans


# sorting by the value
def sort_dict(d):
  return {k: v for k, v in sorted(d.items(), key=lambda item: item[1])}

In [ ]:
## asa_set can be a set of words or set of sentences, same for original_sentence
def get_cos_similarities(asa_set, original_sentence):
  asa_set.insert(0, original_sentence)
  sentence_embeddings = model_ST_bert.encode(asa_set)

  cos_sims = cosine_similarity(
      [sentence_embeddings[0]],
      sentence_embeddings[1:]
  )
  return cos_sims

In [ ]:
def get_important_words_and_sim_scores(original_sentence, combined_set_orig):
  z_freqs = {}  
  orig_sent_POS_tags = get_pos_tags_list(remove_punct(original_sentence))
  POS_tags_dict = {}
  for t in orig_sent_POS_tags[0]:
    POS_tags_dict[t[0]] = t[1]

  original_sentence_arr = remove_punct(original_sentence).split()
  for x in original_sentence_arr:
    if x not in STOP_WORDS and x not in stop_words_nltk:
      print(x.lower(), zipf_frequency(x, 'en'))
      z_freqs[x.lower()] = zipf_frequency(x, 'en')
    ### some correction to keep possessive pronouns, 
    if POS_tags_dict[x] == 'PRP$':
      if x.lower() not in z_freqs:
        print(x, zipf_frequency(x, 'en'))
        z_freqs[x.lower()] = zipf_frequency(x, 'en')

  z_freqs = sort_dict(z_freqs)

  combined_set_copy = list(combined_set_orig).copy()
  combined_set_copy.insert(0, original_sentence)
  bert_sentence_embeddings = model_ST_bert.encode(combined_set_copy)

  cos_sims = cosine_similarity(
      [bert_sentence_embeddings[0]],
      bert_sentence_embeddings[1:]
  )

  cos_sim_list = cos_sims[0]
  sim_dict = {} # map each unaltered sentence to cosine similarity vs the original sentence
  print("original sentence: ", original_sentence)
  i = 0
  sim_col_for_df = []
  for x in list(combined_set_orig).copy():
    sim_dict[x] = cos_sim_list[i]
    sim_col_for_df.append(cos_sim_list[i])
    i = i + 1

  sim_dict = sort_dict(sim_dict) 
  print("  ")
  print("Sorted dict:")
  list_of_sent_and_sims = []
  for h in list(sim_dict.keys()):
    list_of_sent_and_sims.append([h, sim_dict[h]])
  list_of_sent_and_sims.reverse()   # get the similarities rankings from top down (high to low)

  print(list(z_freqs.keys()))

  return list_of_sent_and_sims, z_freqs, sim_col_for_df


In [ ]:
## get paraphrased sentences 
## remove sentences that have retained all the original context w/ added extra words
## look at the sentences that have substituted one or more of the original context words

def get_high_level_sentence_versions(original_sentence, list_of_sent_and_sims, orig_sent_split_arr_lowercase, z_freqs):
  orig_sent_word_counts = get_word_counts(original_sentence) 
  synonyms_dict = {}
  filtered_set_soft = []

  # print("orig_sent_split_arr_lowercase: ", orig_sent_split_arr_lowercase)
  # print("  ")
  test_array = [["David likes to eat peanut butter with a spoon in his mouth", 0]]
  skipover = 0
  # list_of_sent_and_sims = test_array  ### TEST code
  for curr_sent_tup in list_of_sent_and_sims:
    # print("current iteration: ", curr_sent_tup[0])
    curr_sent = curr_sent_tup[0]
    orig_sent_arr_copy = orig_sent_split_arr_lowercase.copy()

    ### check if the curr sentence should be kept
    curr_sent_no_punct_all_lower_split_arr = remove_punct(curr_sent).lower().split()
    # word count
    curr_sent_word_counts = get_word_counts(curr_sent)
    #print(curr_sent_word_counts)
    skipover = 0

    ### if there is some word that was duplicated, skipover it
    #print("curr_sent_word_counts.keys(): ", curr_sent_word_counts.keys())
    for word in list(orig_sent_word_counts.keys()):
      if word not in STOP_WORDS:
        if curr_sent_word_counts[word] > orig_sent_word_counts[word]:
          skipover = 1
    
    # print("skipover: ", skipover)
    ### check if all imporant words are in both
    all_wds_in_both = 1
    for word in list(z_freqs.keys()):
      # print("word to check in both: ", word)
      # print("word in current?: ", word in curr_sent_no_punct_all_lower_split_arr)
      if word not in curr_sent_no_punct_all_lower_split_arr:
          all_wds_in_both = 0

    ## if all important words are in both sentences
    # check if there are any added words in the new sentence
    # print("all_wds_in_both: ", all_wds_in_both)
    if all_wds_in_both == 1:
      for x in curr_sent_no_punct_all_lower_split_arr:
        if x not in list(z_freqs.keys()) and x not in orig_sent_arr_copy :
          skipover = 1

    if skipover == 0:
      filtered_set_soft.append(curr_sent)
      for word in orig_sent_arr_copy: 
        if word in curr_sent_no_punct_all_lower_split_arr:
          curr_sent_no_punct_all_lower_split_arr.remove(word)
          orig_sent_arr_copy.remove(word)
    # print("  ")  
  print("--")
  final_list = []
  for h in filtered_set_soft:
    xs = remove_punct(h)
    if xs not in final_list:
      final_list.append(xs)

  final_list_lowercase = []   # same exact order as final_list
  for c in range (0,len(final_list)):
    final_list_lowercase.append(final_list[c].lower())

  ### showing all sentences that got through the filter
  for c in range (0,len(final_list)):
    print(final_list[c])

  print(len(final_list_lowercase))
  ### get all high level arrangements of the sentence
  print("\n\n")
  orig_sent_no_punct = remove_punct(original_sentence)
  all_sent_arrangements_lower = [orig_sent_no_punct.lower()]
  all_sent_arrangements = [orig_sent_no_punct]
  for ind in range(0, len(final_list_lowercase)):
    curr_sent_arr = final_list_lowercase[ind].split()
    all_wds_in_sent = 1
    # for wd in orig_sent_split_arr_lowercase:
    for wd in list(z_freqs.keys()):
      if wd not in curr_sent_arr:
        all_wds_in_sent = 0
    if all_wds_in_sent == 1:
      if final_list_lowercase[ind] not in all_sent_arrangements_lower:
        all_sent_arrangements_lower.append(final_list_lowercase[ind])
        all_sent_arrangements.append(final_list[ind])

  asa_set = all_sent_arrangements.copy()
  cos_sims = get_cos_similarities(asa_set, original_sentence)

  high_level_arr_and_cos_sims_vs_orig = []
  print("--- all high level sentence arrangements: ---")
  y = 0
  for x in all_sent_arrangements:
    print(x, cos_sims[0][y])
    high_level_arr_and_cos_sims_vs_orig.append([x, cos_sims[0][y]])
    y = y + 1
    
  return filtered_set_soft, high_level_arr_and_cos_sims_vs_orig, all_sent_arrangements_lower, all_sent_arrangements, final_list




In [ ]:
### find out all the matching terms the inference model uses in substitutions
## figure out synsets for the words, return the different level categories of the sentences

def match_like_terms_and_set_levels(original_sentence, f_list, zipff_freqs, all_sent_arrangements):

  zipf_freqs = zipff_freqs.copy()
  high_level_arrangements = all_sent_arrangements.copy()
  print("\n Showing the sentences in order of cos sim for complexity levels:")
  # final_list still has all the punctuation and normal case
  print("Original sentence: ", original_sentence)

  # dictionary of lists: each word maps to a list of terms, with that term's cos sim and zipf
        # zipf of the most important context word if the term has multiple words
  syn_set_with_zipf_sim = {}

  SELECT_PRNNS = ["i", "my", "I", "we", "our"]
  ### making sure we keep important pronouns in
  crucial_context_words = []
  for x in list(zipf_freqs.keys()):
    pos = get_pos_tags_list(x)
    pos = pos[0]
    pos_tag = pos[0][1]
    if x in SELECT_PRNNS:
      crucial_context_words.append(x)
  for x in remove_punct(original_sentence).split():
    pos = get_pos_tags_list(x)
    pos = pos[0]
    pos_tag = pos[0][1]
    if pos_tag == 'NNP':
      crucial_context_words.append(x)

  ### remove the top level arrangements from the final_list
  final_list = f_list.copy()
  for h in high_level_arrangements:
    final_list.remove(h)

  ### also remove sentences that lost the pronouns
  ttt = []
  for x in final_list:
    remove_sent = 0
    # r = remove_punct(x).lower().split() # the correct version, but use later
    # for j in crucial_context_words:
    #   if j not in r:
    for j in crucial_context_words:
      if j not in x:
        remove_sent = 1
    if remove_sent == 1:
      ttt.append(x)
  for x in ttt:
    final_list.remove(x)

  ff_list = final_list.copy()
  cos_sims_final_list = get_cos_similarities(ff_list, original_sentence)
  cos_sims_final_list = cos_sims_final_list[0]  ## should be same length as final_list
  
  sent_to_sim_dict = {}
  print(len(final_list))
  #  
  for x in range(0, len(final_list)):
    sent_to_sim_dict[final_list[x]] = cos_sims_final_list[x]
    print(final_list[x], " || sim_score: ", cos_sims_final_list[x])
  
  #########################################
  #### NOW TO DO THE SYN SET CREATION  ####
  #########################################

  ### prepping for term matching, list of lists
  top_level_no_punct_all_lower_split_arrs = []
  for g in high_level_arrangements:
    q = remove_punct(g).lower().split()
    top_level_no_punct_all_lower_split_arrs.append(q)
  
  ### first, we need to find all the like terms and put them in the dict of lists
  ### next, we find the cut off points for our sentence levels
  ### iterate through the final_list until you find the first sentence
  ## that has completely removed one context word/phrase from the original
  ### The next level down will be filled with those sentences that have only subs
  ### and so on
  
  ### decide on either soft constraints or hard_constraints
  ### These 2 are to hold the more complex words that might appear (and have no synonyms created)
  soft_constraint_context_set, hard_constraint_context_set = get_important_word_stats(original_sentence, zipf_freqs)
  
  all_crucial_context_words_soft = crucial_context_words + soft_constraint_context_set
  all_crucial_context_words_hard = crucial_context_words + hard_constraint_context_set

  ### for each sentence
  # the idea is to iteratively remove all the shared context words from both the original 
  ## and the current sentence, and see if there are any words left that could be like terms
  ## this is done by removing shared terms from both top level and current sentence
  ## max of 2 context words can remain on the top level
  ## then lump up the remaining words in the bottom, doing token splits if needed, and match them
  ## against the context words in the top level
  ## key note: remember to split the sentence at the determiners to avoid clumping,
  ##        since those are the ones most likely to be lost in paraphrasing 
  ###       trick: if you have a shared determiner, replace it with qqzqq in the current sentence
  ###       then later when you do .split(), remember to separate the sentence at the qqzqq

  ## the actual sorting of sentences into their respective levels
  sents_top_level = high_level_arrangements # no context lost, no substitutions
  sents_one_lvl_down = [] # sentences with one or two substitutions
  sents_two_lvls_down = [] # subsitutions and losing ONE context word/phrases
  sents_three_lvls_down = [] # the rest (either too many subs or too much lost context)
  max_zipf_val = np.max(list(zipf_freqs.values()))

  ### crude dict of which determiners are attached to which context words:
  dets_per_context_word = {}
  orig_temp = remove_punct(original_sentence).lower().split()
  for i in range(0, len(orig_temp)-1):
    if orig_temp[i] in MAIN_DETS:
      if orig_temp[i+1] in list(zipf_freqs.keys()):
        if orig_temp[i+1] not in dets_per_context_word:
          dets_per_context_word[i+1] = [orig_temp[i]]
        else:
          arr = dets_per_context_word[i+1]
          dets_per_context_word[i+1] = arr.append(orig_temp[i])

  SYNSET_OF_CONTEXT_WORDS = {}
  stop_wd_subset = ["a", "an", "the", "and", "that", "it", "for", "or", "but", "in", "of","not"]
  SPLIT_TOKEN = "qqzqq"
  KEEP = "qqqzzqqq"

  print("crucial_context_words: ", crucial_context_words)

  tlnpalsarrs = top_level_no_punct_all_lower_split_arrs.copy()
  sent_top = tlnpalsarrs[0]
  tlnp = []
  for n in range(0, len(final_list)):
    tlnp.append(sent_top.copy())
  # final_list = ["david likes to eat a spoonful of peanut butter with a spoonful"] # test code
  for i in range(0, len(final_list)):
    sent = final_list[i]
    ### split the sentence into a word array
    curr_sent_chunks = []  
    curr_sent_chunks.append(remove_punct(sent).lower().split())  

    ## top level 
    

    ### first, check if there are any crucial context words missing:
    has_all_crucial_context = 1
    for x in crucial_context_words:
      if x.lower() not in curr_sent_chunks[0]:
        has_all_crucial_context = 0
    # print("iteration: {}".format(i))
    # print("this sentence: ", sent)
    # print("has_all_crucial_context: ", has_all_crucial_context)
    if has_all_crucial_context == 0:
      continue

    sent_one = tlnp[i]
    # print("sentence top level: ", sent_one)
    # print("current sentence: ", curr_sent_chunks[0])
    chunk = curr_sent_chunks[0]
    sent_one_copy = sent_one.copy()
    for wd in sent_one_copy:
      # print("is context word [", wd, "] in current sentence: ", wd in chunk)
      if wd in chunk:
        chunk.remove(wd)
        sent_one.remove(wd)
    #     print("top level sentence after removal: ", sent_one)
    #     print("current sentence after removal: ", chunk)
    # print("\n what's left:")
    # print("top level sentence afer matching: ", sent_one)
    # print("current sentence after matching: ", chunk)
    remaining_words = chunk
    # print(SYNSET_OF_CONTEXT_WORDS)
    ### now we check the remaining words left in current sentence
    ###splitting process, hopefully we will have synonym sets already found by the inference model,
    ### else, we just leave it in the lowest level group
    ### if the bottom level is empty/only useless words, either all matched or missing context
    result = []
    if len(remaining_words) > 0:
      ### make sure it's not just only stop words left in the bottom level before combining the word
      all_stop_wds = 1
      for wd in remaining_words:
        if wd not in STOP_WORDS:
          all_stop_wds = 0
      if all_stop_wds == 0:
        str_together = remaining_words[0]
        i = 1
        while i < len(remaining_words):
          str_together = str_together + " " + remaining_words[i]
          i = i + 1
      else:
        str_together = ""
    else:
      str_together = ""
    
    for x in sent_one:
      if x in STOP_WORDS:
        sent_one.remove(x)
    
    # nothing to check, maybe some minor change to the original
    if len(sent_one) == 0:
      sents_one_lvl_down.append(sent)

    # only one sub or one context lost
    elif len(sent_one) == 1:  
      if str_together == "":
        sents_two_lvls_down.append(sent)
      else:
        this_word = sent_one[0]
        # print("this_word: ", this_word)
        # print("curr sent left: ", str_together)
        # bb = this_word not in list(SYNSET_OF_CONTEXT_WORDS.keys())
        # print("this_word not in list(SYNSET_OF_CONTEXT_WORDS.keys(): ", bb)
        # print("SYNSET_OF_CONTEXT_WORDS before operations: ", SYNSET_OF_CONTEXT_WORDS)
        if this_word not in list(SYNSET_OF_CONTEXT_WORDS.keys()):
          SYNSET_OF_CONTEXT_WORDS[this_word] = [str_together]
        else:
          # temp = SYNSET_OF_CONTEXT_WORDS[this_word].copy()
          SYNSET_OF_CONTEXT_WORDS[this_word].append(str_together)
        sents_one_lvl_down.append(sent)
        # print("SYNSET_OF_CONTEXT_WORDS AFTER operations: ", SYNSET_OF_CONTEXT_WORDS)
    ## three cases
    elif len(sent_one) == 2:   
      if str_together == "":  ## zero syns in the bottom case
        sents_three_lvls_down.append(sent)
      else: ## one or two syns in the bottom
        for con_word in sent_one:  
          # remove synonyms we already before
          if con_word in list(SYNSET_OF_CONTEXT_WORDS.keys()):
            for possible_syn in SYNSET_OF_CONTEXT_WORDS[con_word]:
              if possible_syn in str_together:
                str_together = str_together.replace(str_together, "")
                str_together = str_together.replace("  ", " ")
                str_together = str_together.strip()
                sent_one.remove(con_word)  ## removed the word from top level, and syn from bottom level
        
        ### Hardest case:
        ### if we have 2 context words left somehow (unlikely), and unknown amount of tokens in str_together
        ## try multiple splits on str_together, run them through the similarity checker to find the 
        ### best split on the remaning curr sentence words for both context words
        if len(sent_one) == 2:
          asdf =7
        if len(sent_one) == 1:  ## was able to remove a pre-existing synonym/similar phrase
          if str_together == "": ## one context lost
            sents_two_lvls_down.append(sent)
          else: ## no context lost, new syn found
            this_word = sent_one[0]
            if this_word not in list(SYNSET_OF_CONTEXT_WORDS.keys()):
              SYNSET_OF_CONTEXT_WORDS[this_word] = [str_together]
            else:
              temp = SYNSET_OF_CONTEXT_WORDS[this_word].copy()
              SYNSET_OF_CONTEXT_WORDS[this_word] = temp.append(str_together)
            sents_one_lvl_down.append(sent)

    ## too much still remaining in top level, either too many subs had to happen, or too much context lost
    else:  
      sents_three_lvls_down.append(sent)

    ## after all common words removed from both current and top level
    ### check how many important context words are lost in the current sentence
    # this is done by checking the top level ones:
    # if there is 1 context word left in the top level arrangements, and nothing but uesless words
    ## in the current sentence, then 1 context word was lost
    ## if you can match all the tokens in the current sentence, then no context was lost

  return sents_top_level, sents_one_lvl_down, sents_two_lvls_down, sents_three_lvls_down, SYNSET_OF_CONTEXT_WORDS


  


In [ ]:
def simplify_sentence(original_sentence):
  orig_sentence_no_punct = remove_punct(original_sentence)
  orig_sentence_no_punct_all_lower = orig_sentence_no_punct.lower()

  orig_sent_no_punct_split_arr = orig_sentence_no_punct.split()
  orig_sent_no_punct_all_lower_split_arr = orig_sentence_no_punct_all_lower.split()

  final_sent_orig = orig_sentence_no_punct  #original sentence with no punctuation 

  set_1 = set(get_paraphrased_sentences(model_pega, tokenizer_pega, original_sentence, num_beams=20, num_return_sequences=20))
  set_2 = set(get_paraphrased_sentences(model_paws, tokenizer_paws, original_sentence, num_beams=20, num_return_sequences=20))
  set_3 = set(get_paraphrased_sentences(model_parr, tokenizer_parr, original_sentence, num_beams=20, num_return_sequences=20))
  set_x = set(get_paraphrased_sentences(model_ctrl44, tokenizer_ctrl44, original_sentence, num_beams=20, num_return_sequences=20))
  set_xy = set(get_paraphrased_sentences(model_ctrl44, tokenizer_ctrl44, "<para> "+original_sentence, num_beams=20, num_return_sequences=20))

  id_tags = []
  k = 0
  full_set_orig = []  # with punctuation all normal case
  full_set_no_punct = []  # with no punctuation, all normal case
  full_set_no_punct_all_lower = [] # with no punctuation, all lower case

  combined_set_orig = set_1.union(set_2).union(set_3).union(set_x).union(set_xy)
  # print("all possible sentenecs: ")
  for f in combined_set_orig:
    # print(f)
    full_set_orig.append(f)
    full_set_no_punct.append(remove_punct(f))
    full_set_no_punct_all_lower.append(remove_punct(f).lower())
    id_tags.append(k)
    k = k + 1
  print(" ")

  d = {'SentID': id_tags, 'OrigSent': full_set_orig, 'NoPunctSent': full_set_no_punct, 'NoPunctAllLower': full_set_no_punct_all_lower}
  sentences_df = pd.DataFrame(data=d)
  full_sentence_df = sentences_df.copy(deep=True)  # holds all the sentences and tags
  print("number of sentences", len(full_sentence_df))
  
  list_of_sent_and_sims, z_freqs, sim_col_for_df = get_important_words_and_sim_scores(original_sentence, combined_set_orig)
  full_sentence_df['Cos_Sim_vs_Orig_Sent'] = sim_col_for_df
  filtered_set_soft, high_level_arr_and_cos_sims_vs_orig, all_sent_arrangements_lower, all_sent_arrangements, final_list = get_high_level_sentence_versions(original_sentence, list_of_sent_and_sims, orig_sent_no_punct_all_lower_split_arr, z_freqs)
  sents_top_level, sents_one_lvl_down, sents_two_lvls_down, sents_three_lvls_down, syndict = match_like_terms_and_set_levels(original_sentence, final_list, z_freqs, all_sent_arrangements)
  
  print("   \n\n\n")
  print("Results:")
  print("Level 0: ")
  for x in sents_top_level:
    print(x)
  print("\n")
  print("Level -1: ")
  for x in sents_one_lvl_down:
    print(x)
  print("\n")
  print("Level -2: ")
  for x in sents_two_lvls_down:
    print(x)
  print("\n")
  print("Level -3: ")
  for x in sents_three_lvls_down:
    print(x)
  print("\n")

  PREPS_COMMON = ["with", "at", "by", "to", "in", "for", "from", "of", "on"]

  print("Synsets Breakdown: ")
  for k in list(syndict.keys()):
    print("Word: ", k)
    print("----------------------------")
    t = list(set(syndict[k]))
    syns_in_order_of_sim = {}
    syns_listing = []
    for y in t:
      if len(y.split()) == 2:
        syn_split = y.split()
        pos_list = get_pos_tags_list(y)[0]
        first_term = pos_list[0][1]
        second_term = pos_list[1][1]
        if first_term in list(MAIN_POS_TAGS.keys()) and second_term in list(MAIN_POS_TAGS.keys()):
          if MAIN_POS_TAGS[second_term] == "verb" and MAIN_POS_TAGS[first_term] != 'adverb':
            temp_list = []
            for ws in PREPS_COMMON:
              temp_list.append(syn_split[0] + " " + ws + " " + syn_split[1])
            temp_temp_list = temp_list.copy()
            cos_sims_preps_added = get_cos_similarities(temp_temp_list, k)
            cos_sims_preps_added = cos_sims_preps_added[0]
            phrase_dict_sims = {}
            for ind in range(0, len(temp_list)):
              phrase_dict_sims[temp_list[ind]] = cos_sims_preps_added[ind]
            phrase_dict_sims = sort_dict(phrase_dict_sims)
            best_key = list(phrase_dict_sims.keys())[0]
            syns_listing.append(best_key)
      else:
        syns_listing.append(y)
    temp_syns_listing = syns_listing.copy()
    sims_of_syns = get_cos_similarities(temp_syns_listing, k)
    sims_of_syns = sims_of_syns[0]
    for ind in range(0, len(syns_listing)):
      syns_in_order_of_sim[syns_listing[ind]] = sims_of_syns[ind]
    syns_in_order_of_sim = sort_dict(syns_in_order_of_sim)
    tt = list(syns_in_order_of_sim.keys())
    for sind in range(len(tt)-1, -1, -1):
      synonym = tt[sind]
      print("\t",synonym)
    print(" ")


    ### last step: add syns for words that aren't in spacy medium using nltk SynSets


In [ ]:
print(gunning_fog("I photographed"))
print(gunning_fog("I took a picture"))
print(gunning_fog("I took a photo"))

print(" ")
print(zipf_frequency("photographed", 'en'))
print(zipf_frequency("picture", 'en'))
print(zipf_frequency("photo", 'en'))

print(get_cos_similarities(["I took a picture", "I took a photo"], "I photographed"))

print(zipf_frequency("moisture", 'en'))


print(get_cos_similarities(["appear from disappear", "appear to disappear", "appear by disappear"], "vanish"))

In [ ]:
print(zipf_frequency("discretion", 'en'))

3.9


In [ ]:
all_zipfs_in_spacy = []
for x in nlp_medium_words:
  all_zipfs_in_spacy.append(zipf_frequency(x, 'en'))

print(np.mean(all_zipfs_in_spacy))

1.5555920326503698


In [ ]:
print(np.quantile(all_zipfs_in_spacy, .957))  ### and this one
print(np.quantile(all_zipfs_in_spacy, .90))  ### probably use this one
print(np.quantile(all_zipfs_in_spacy, .897))

### in between .95 and .9 are harder words that may be known

4.24
3.38
3.35


In [ ]:
tokens = nlp_medium("photograph photo picture image")
  
for token in tokens:
    # Printing the following attributes of each token.
    # text: the word string, has_vector: if it contains
    # a vector representation in the model, 
    # vector_norm: the algebraic norm of the vector,
    # is_oov: if the word is out of vocabulary.
    print(token.text, token.has_vector, token.vector_norm, token.is_oov)

for t1 in range(0, len(tokens)):
  token1 = tokens[t1]
  for t2 in range(t1, len(tokens)):
    token2 = tokens[t2]
    print("Similarity: {}, {}".format(token1, token2), token1.similarity(token2))

print(get_cos_similarities(["photo", "picture", "image"], "photograph"))


get_pos_tags_list("Miles ate a hamburger")

In [ ]:
### if no synonyms generated from the inference models, or if we want another option, switch to Synsets
# substitute = get_similar_words("prolixity", "noun")
sgsg = get_similar_words("reticence", "noun")
print("sub: ", sgsg)

Test out Our Program!

In [ ]:
h = input() 
simplify_sentence(h)

